In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jan 15 11:37:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from enum import Enum
import imageio
import os
import hashlib

%matplotlib inline

dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [2]:
################################################################################
# DATASETS
################################################################################


class Dataset:
    def __init__(self, X_train, y_train, X_test, y_test, shape, shape_flattened):
        X_train = X_train.astype(dtype) / 255.0
        y_train = y_train.astype(dtype)
        X_test = X_test.astype(dtype)  / 255.0
        y_test = y_test.astype(dtype)

        X_train = np.reshape(X_train, shape_flattened)
        X_test = np.reshape(X_test, shape_flattened)

        X = np.concatenate((X_train, X_test), axis=0)
        y = np.concatenate((y_train, y_test), axis=0)

        from sklearn.preprocessing import StandardScaler

        scaler = StandardScaler()
        scaler.fit(X_train)  # Scaling each feature independently

        X_norm = scaler.transform(X)
        X_train_norm = scaler.transform(X_train)
        X_test_norm = scaler.transform(X_test)

        X_norm = np.reshape(X_norm, shape)
        X_train_norm = np.reshape(X_train_norm, shape)
        X_test_norm = np.reshape(X_test_norm, shape)

        self.X_norm = X_norm
        self.y = y
        self.X_train_norm = X_train_norm
        self.y_train = y_train
        self.X_test_norm = X_test_norm
        self.y_test = y_test


def get_cifar_10_dataset():
    cifar10 = tf.keras.datasets.cifar10
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    shape = (-1, 32, 32, 3)
    shape_flattened = (-1, 3072)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


def get_cifar_100_dataset():
    cifar100 = tf.keras.datasets.cifar100
    (X_train, y_train), (X_test, y_test) = cifar100.load_data()
    shape = (-1, 32, 32, 3)
    shape_flattened = (-1, 3072)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


def get_svhn_dataset():
    from urllib.request import urlretrieve
    from scipy import io

    train_filename, _ = urlretrieve('http://ufldl.stanford.edu/housenumbers/train_32x32.mat')
    test_filename, _ = urlretrieve('http://ufldl.stanford.edu/housenumbers/test_32x32.mat')

    X_train = io.loadmat(train_filename, variable_names='X').get('X')
    y_train = io.loadmat(train_filename, variable_names='y').get('y')
    X_test = io.loadmat(test_filename, variable_names='X').get('X')
    y_test = io.loadmat(test_filename, variable_names='y').get('y')

    X_train = np.moveaxis(X_train, -1, 0)
    y_train -= 1
    X_test = np.moveaxis(X_test, -1, 0)
    y_test -= 1

    shape = (-1, 32, 32, 3)
    shape_flattened = (-1, 3072)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


def get_tiny_imagenet_dataset():
    """
    Original source: https://github.com/sonugiri1043/Train_ResNet_On_Tiny_ImageNet/blob/master/Train_ResNet_On_Tiny_ImageNet.ipynb
    Original author: sonugiri1043@gmail.com
    """

    if not os.path.isdir('IMagenet'):
        ! git clone https://github.com/seshuad/IMagenet

    print("Processing the downloaded dataset...")

    path = 'IMagenet/tiny-imagenet-200/'

    id_dict = {}
    for i, line in enumerate(open(path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i

    train_data = list()
    test_data = list()
    train_labels = list()
    test_labels = list()

    for key, value in id_dict.items():
        train_data += [imageio.imread(path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), pilmode='RGB') for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open(path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(imageio.imread(path + 'val/images/{}'.format(img_name), pilmode='RGB'))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    X_train = np.array(train_data)
    y_train = np.argmax(np.array(train_labels), axis=1)
    X_test = np.array(test_data)
    y_test = np.argmax(np.array(test_labels), axis=1)

    shape = (-1, 64, 64, 3)
    shape_flattened = (-1, 12288)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


def get_mnist_dataset():
    mnist = tf.keras.datasets.mnist
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    shape = (-1, 28, 28, 1)
    shape_flattened = (-1, 1)  # Scaling all features together
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


def get_fashion_mnist_dataset():
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    shape = (-1, 28, 28, 1)
    shape_flattened = (-1, 1)  # Scaling all features together
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


################################################################################
# REGULARIZERS
################################################################################


class Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self):
        self.n_new_neurons = 0
        self.scaling_tensor = None
        self.set_regularization_penalty(0.)
        self.set_regularization_method(None)

    def __call__(self, x):
        if self.regularization_method is None or self.regularization_penalty == 0:
            return 0
        elif self.regularization_method == 'weighted_l1':
            return self.weighted_l1(x)
        elif self.regularization_method == 'weighted_l1_reordered':
            return self.weighted_l1_reordered(x)
        elif self.regularization_method == 'group_sparsity':
            return self.group_sparsity(x)
        elif self.regularization_method == 'l1':
            return self.l1(x)
        else:
            raise NotImplementedError(f"Unknown regularization method {self.regularization_method}")
    
    def weighted_l1(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # the scaling tensor, as well as the resulting weighted values, could be:
        #
        # [[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]]
        #
        # Therefore every additional output neuron is regularized more.

        scaling_tensor = tf.cumsum(tf.constant(self.regularization_penalty, shape=x.shape, dtype=dtype), axis=-1)
        weighted_values = scaling_tensor * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def weighted_l1_reordered(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # the scaling tensor, as well as the resulting weighted values, could be:
        #
        # [[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]]
        #
        # Therefore every additional output neuron is regularized more.

        if self.update_scaling_tensor:
            scaling_tensor_raw = tf.cumsum(tf.constant(self.regularization_penalty, shape=x.shape, dtype=dtype), axis=-1)

            scaling_tensor_old_neurons = scaling_tensor_raw[:, :-self.n_new_neurons]
            scaling_tensor_new_neurons = scaling_tensor_raw[:, -self.n_new_neurons:]
            scaling_tensor_old_neurons_shuffled = tf.transpose(tf.random.shuffle(tf.transpose(scaling_tensor_old_neurons)))
            self.scaling_tensor = tf.concat([scaling_tensor_old_neurons_shuffled, scaling_tensor_new_neurons], axis=-1)
            self.update_scaling_tensor = False

        weighted_values = self.scaling_tensor * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def group_sparsity(self, x):
        # I.e. for a parameter matrix of 3 input and 5 output neurons:
        #
        # [[1., 1., 1., 1., 1.],
        #  [1., 2., 2., 1., 2.],
        #  [2., 2., 3., 1., 3.]]
        #
        # The resulting vector of group norms is [2.4494896, 3.       , 3.7416573, 1.7320508, 3.7416573], 
        # therefore for every output neuron, its incoming connections form a group.

        group_norms = tf.norm(x, ord=2, axis=0)
        # assert group_norms.shape[0] == x.shape[1]
        return self.regularization_penalty * tf.reduce_sum(group_norms)
    
    def l1(self, x):
        weighted_values = self.regularization_penalty * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def prune(self):
        self.n_new_neurons = 0
        if self.regularization_method == 'weighted_l1_reordered':
            self.update_scaling_tensor = True
    
    def grow(self, n_new_neurons):
        self.n_new_neurons = n_new_neurons
        if self.regularization_method == 'weighted_l1_reordered':
            self.update_scaling_tensor = True
    
    def set_regularization_penalty(self, regularization_penalty):
        self.regularization_penalty = regularization_penalty
    
    def set_regularization_method(self, regularization_method):
        self.regularization_method = regularization_method
        if self.regularization_method == 'weighted_l1_reordered':
            self.update_scaling_tensor = True
        else:
            self.update_scaling_tensor = None

    def get_config(self):
        return {'regularization_penalty': float(self.regularization_penalty)}


################################################################################
# LAYERS
################################################################################


class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, input_shape):
        super().__init__()

        self.inpt_shp = input_shape


class Dense(CustomLayer):
    def __init__(self, units, activation, kernel_initializer='glorot_uniform', 
                 bias_initializer='zeros', input_shape=None, fixed_size=False):
        super().__init__(input_shape)

        self.units = units
        self.activation = activation
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.fixed_size = fixed_size
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = Regularizer()
    
    def build(self, input_shape):
        input_units = input_shape[-1]

        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, self.units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(self.units,), dtype=dtype),
            trainable=True)
        
        self.add_loss(lambda: self.regularizer(tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs, training=None):
        return self.A(tf.matmul(inputs, self.W) + self.b)

    def get_size(self):
        return self.W.shape[0], self.W.shape[1]
    
    def prune(self, threshold, active_input_units_indices):
        # Remove connections from pruned units in previous layer
        new_W = tf.gather(self.W.value(), active_input_units_indices, axis=0)

        if self.fixed_size:
            active_output_neurons_indices = list(range(new_W.shape[1]))
        else:
            # Prune units in this layer
            weights_with_biases = tf.concat([new_W, tf.reshape(self.b.value(), (1, -1))], axis=0)
            neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
            active_output_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W = tf.gather(new_W, active_output_neurons_indices, axis=1)
            new_b = tf.gather(self.b.value(), active_output_neurons_indices, axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.W = tf.Variable(name='W', initial_value=new_W, trainable=True)

        self.regularizer.prune()
        return active_output_neurons_indices
    
    def grow(self, n_new_input_units, percentage, min_new_units, scaling_factor):
        if n_new_input_units > 0:
            # Add connections to grown units in previous layer
            W_growth = self.W_init(shape=(self.W.shape[0] + n_new_input_units, self.W.shape[1]), dtype=dtype)[-n_new_input_units:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.
            new_W = tf.concat([self.W.value(), W_growth], axis=0)
        else:
            new_W = self.W.value()

        if self.fixed_size:
            n_new_output_units = 0
        else:
            # Grow new units in this layer
            n_new_output_units = max(min_new_units, int(new_W.shape[1] * percentage))
            if n_new_output_units > 0:
                W_growth = self.W_init(shape=(new_W.shape[0], new_W.shape[1] + n_new_output_units), dtype=dtype)[:, -n_new_output_units:] * scaling_factor
                b_growth = self.b_init(shape=(n_new_output_units,), dtype=dtype)  # TODO for all possible bias initializers to work properly, the whole bias vector should be initialized at once
                new_W = tf.concat([new_W, W_growth], axis=1)
                new_b = tf.concat([self.b.value(), b_growth], axis=0)

                self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.W = tf.Variable(name='W', initial_value=new_W, trainable=True)

        self.regularizer.grow(n_new_output_units)
        return n_new_output_units
    
    def set_regularization_penalty(self, regularization_penalty):
        if not self.fixed_size:
            self.regularizer.set_regularization_penalty(regularization_penalty)
    
    def set_regularization_method(self, regularization_method):
        if not self.fixed_size:
            self.regularizer.set_regularization_method(regularization_method)
    
    def get_param_string():
        param_string = ""
        weights_with_bias = tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)
        max_parameters = tf.math.reduce_max(tf.abs(weights_with_bias), axis=0).numpy()
        magnitudes = np.floor(np.log10(max_parameters))
        for m in magnitudes:
            if m > 0:
                m = 0
            param_string += str(int(-m))
        return param_string


class Conv2D(CustomLayer):
    def __init__(self, filters, filter_size, activation, strides=(1, 1), 
                 padding='SAME', kernel_initializer='glorot_uniform',
                 bias_initializer='zeros', input_shape=None, fixed_size=False):
        super().__init__(input_shape)
    
        self.filters = filters
        self.filter_size = filter_size
        self.activation = activation
        self.strides = strides
        self.padding = padding
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.fixed_size = fixed_size
        
        self.A = tf.keras.activations.get(activation)
        self.F_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = Regularizer()
    
    def build(self, input_shape):
        input_filters = input_shape[-1]

        self.F = tf.Variable(
            name='F',
            initial_value=self.F_init(
                shape=(self.filter_size[0], self.filter_size[1], input_filters, self.filters), dtype=dtype
            ),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(self.filters,), dtype=dtype),
            trainable=True)

        self.add_loss(lambda: self.regularizer(tf.concat([tf.reshape(self.F, (-1, self.F.shape[-1])), tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs, training=None):
        y = tf.nn.conv2d(inputs, self.F, strides=self.strides, padding=self.padding)
        y = tf.nn.bias_add(y, self.b)
        y = self.A(y)
        return y
    
    def get_size(self):
        return self.F.shape[-2], self.F.shape[-1]
    
    def prune(self, threshold, active_input_units_indices):
        # Remove connections from pruned units in previous layer
        new_F = tf.gather(self.F.value(), active_input_units_indices, axis=-2)

        if self.fixed_size:
            active_output_filters_indices = list(range(new_F.shape[-1]))
        else:
            # Prune units in this layer
            F_reduced_max = tf.reshape(tf.math.reduce_max(tf.abs(new_F), axis=(0, 1, 2)), (1, -1))
            F_reduced_max_with_biases = tf.concat([F_reduced_max, tf.reshape(self.b.value(), (1, -1))], axis=0)
            filters_are_active = tf.math.reduce_max(tf.abs(F_reduced_max_with_biases), axis=0) >= threshold
            active_output_filters_indices = tf.reshape(tf.where(filters_are_active), (-1,))
            
            new_F = tf.gather(new_F, active_output_filters_indices, axis=-1)
            new_b = tf.gather(self.b.value(), active_output_filters_indices, axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.F = tf.Variable(name='F', initial_value=new_F, trainable=True)

        self.regularizer.prune()
        return active_output_filters_indices

    def grow(self, n_new_input_units, percentage, min_new_units, scaling_factor):
        if n_new_input_units > 0:
            # Add connections to grown units in previous layer
            F_growth = self.F_init(shape=(self.F.shape[0], self.F.shape[1], self.F.shape[2] + n_new_input_units, self.F.shape[3]), dtype=dtype)[:, :, -n_new_input_units:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.
            new_F = tf.concat([self.F.value(), F_growth], axis=-2)
        else:
            new_F = self.F.value()

        if self.fixed_size:
            n_new_output_units = 0
        else:
            # Grow new units in this layer
            n_new_output_units = max(min_new_units, int(new_F.shape[-1] * percentage))
            if n_new_output_units > 0:
                F_growth = self.F_init(shape=(new_F.shape[0], new_F.shape[1], new_F.shape[2], new_F.shape[3] + n_new_output_units), dtype=dtype)[:, :, :, -n_new_output_units:] * scaling_factor
                b_growth = self.b_init(shape=(n_new_output_units,), dtype=dtype)  # TODO for all possible bias initializers to work properly, the whole bias vector should be initialized at once
                new_F = tf.concat([new_F, F_growth], axis=-1)
                new_b = tf.concat([self.b.value(), b_growth], axis=0)

                self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.F = tf.Variable(name='F', initial_value=new_F, trainable=True)

        self.regularizer.grow(n_new_output_units)
        return n_new_output_units
    
    def set_regularization_penalty(self, regularization_penalty):
        if not self.fixed_size:
            self.regularizer.set_regularization_penalty(regularization_penalty)
    
    def set_regularization_method(self, regularization_method):
        if not self.fixed_size:
            self.regularizer.set_regularization_method(regularization_method)

    def get_param_string():
        param_string = ""
        # TODO
        return param_string


class Flatten(tf.keras.Model):
    def call(self, inputs, training=None):
        return tf.reshape(tf.transpose(inputs, perm=[0, 3, 1, 2]), (inputs.shape[0], -1))


################################################################################
# MODELS
################################################################################


class Epoch:
    def __init__(self, grow, prune, regularization_penalty, regularization_method):
        self.grow = grow
        self.prune = prune
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
    
    def __str__(self):
        return f'{int(self.grow)}{int(self.prune)}{self.regularization_penalty}{self.regularization_method}'
    
    def __repr__(self):
        return self.__str__()


class DynamicEpoch(Epoch):
    def __init__(self, regularization_penalty, regularization_method):
        super().__init__(True, True, regularization_penalty, regularization_method)


class StaticEpoch(Epoch):
    def __init__(self, regularization_penalty, regularization_method):
        super().__init__(False, False, regularization_penalty, regularization_method)


class StaticEpochNoRegularization(StaticEpoch):
    def __init__(self):
        super().__init__(0., None)


class Schedule:
    def __init__(self, epochs):
        self.epochs = epochs

    def __iter__(self):
        return self.epochs.__iter__()
    
    def __len__(self):
        return len(self.epochs)
    
    def __str__(self):
        text = ''.join([str(epoch) for epoch in self.epochs])
        return hashlib.sha1(text.encode('utf-8')).hexdigest()[:10]
    
    def __repr__(self):
        return self.__str__()


class Sequential(tf.keras.Model):
    def __init__(self, layers, activation=None):
        super().__init__()
        
        self.lrs = layers
        
    def call(self, inputs, training=None):
        x = inputs
        for layer in self.lrs:
            x = layer(x, training=training)
        return x
    
    def get_layer_input_shape(self, target_layer):
        if target_layer.inpt_shp is not None:
            return target_layer.inpt_shp

        input = np.random.normal(size=(1,) + self.lrs[0].inpt_shp)
        for layer in self.lrs:
            if layer is target_layer:
                return tuple(input.shape[1:])
            input = layer(input)
        raise Exception("Layer not found in the model.")

    def get_layer_output_shape(self, target_layer):
        input = np.random.normal(size=(1,) + self.lrs[0].inpt_shp)
        for layer in self.lrs:
            output = layer(input)
            if layer is target_layer:
                return tuple(output.shape[1:])
            input = output
        raise Exception("Layer not found in the model.")
    
    def get_layer_sizes(self):
        """
        Returns the sizes of all layers in the model, including the input and output layer.
        """
        layer_sizes = list()
        first_layer = True
        for l in range(len(self.lrs)):
            layer = self.lrs[l]
            if isinstance(layer, CustomLayer):
                layer_size = layer.get_size()
                if first_layer:
                    layer_sizes.append(layer_size[0])
                    first_layer = False
                layer_sizes.append(layer_size[1])
        return layer_sizes
    
    def get_hidden_layer_sizes(self):
        return self.get_layer_sizes()[1:-1]
    
    def get_regularization_penalty(self):
        #TODO improve
        return self.lrs[-2].regularizer.regularization_penalty
    
    def set_regularization_penalty(self, regularization_penalty):
        for layer in self.lrs:
            if isinstance(layer, CustomLayer) and not layer.fixed_size:
                layer.set_regularization_penalty(regularization_penalty)
    
    def set_regularization_method(self, regularization_method):
        for layer in self.lrs:
            if isinstance(layer, CustomLayer) and not layer.fixed_size:
                layer.set_regularization_method(regularization_method)

    def prune(self, params):
        input_shape = self.get_layer_input_shape(self.lrs[0])
        n_input_units = input_shape[-1]
        active_units_indices = list(range(n_input_units))

        last_custom_layer = None
        for layer in self.lrs:
            if isinstance(layer, CustomLayer):
                if last_custom_layer is not None and type(last_custom_layer) != type(layer):
                    if type(last_custom_layer) == Conv2D and type(layer) == Dense:
                        convolutional_shape = self.get_layer_output_shape(last_custom_layer)
                        active_units_indices = self.convert_channel_indices_to_flattened_indices(active_units_indices, convolutional_shape)
                    else:
                        raise Exception("Incorrect order of custom layer types.")
                active_units_indices = layer.prune(params.pruning_threshold, active_units_indices)
                last_custom_layer = layer
    
    def grow(self, params):   
        n_new_units = 0

        last_custom_layer = None
        for layer in self.lrs:
            if isinstance(layer, CustomLayer):
                if last_custom_layer is not None and type(last_custom_layer) != type(layer):
                    if type(last_custom_layer) == Conv2D and type(layer) == Dense:
                        convolutional_shape = self.get_layer_output_shape(last_custom_layer)
                        n_new_units = n_new_units * convolutional_shape[0] * convolutional_shape[1]
                    else:
                        raise Exception("Incorrect order of custom layer types.")
                n_new_units = layer.grow(n_new_units, params.growth_percentage, min_new_units=params.min_new_neurons, scaling_factor=params.pruning_threshold)
                last_custom_layer = layer
    
    @staticmethod
    def convert_channel_indices_to_flattened_indices(channel_indices, convolutional_shape):
        dense_indices = list()
        units_per_channel = convolutional_shape[0] * convolutional_shape[1]
        for channel_index in channel_indices:
            for iter in range(units_per_channel):
                dense_indices.append(channel_index * units_per_channel + iter)
        return dense_indices
    
    def print_neurons(self):
        for layer in self.lrs[:-1]:
            print(layer.get_param_string())
    
    def evaluate(self, params, summed_training_loss, summed_training_accuracy):
        # Calculate training loss and accuracy
        if summed_training_loss is not None:
            loss = summed_training_loss / params.x.shape[0]
        else:
            loss = None
        
        if summed_training_accuracy is not None:
            accuracy = summed_training_accuracy / params.x.shape[0]
        else:
            accuracy = None
        
        # Calculate val loss and accuracy
        summed_val_loss = 0
        summed_val_accuracy = 0
        n_val_instances = 0
        
        for step, (x_batch, y_batch) in enumerate(params.val_dataset):
            y_pred = self(x_batch, training=False)
            summed_val_loss += tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
            summed_val_accuracy += float(tf.reduce_sum(tf.keras.metrics.sparse_categorical_accuracy(y_batch, y_pred)))
            n_val_instances += x_batch.shape[0]
        
        val_loss = summed_val_loss / n_val_instances
        val_accuracy = summed_val_accuracy / n_val_instances

        return loss, accuracy, val_loss, val_accuracy
    
    def count_params(self):
        trainable_count = np.sum([K.count_params(w) for w in self.trainable_weights])
        non_trainable_count = np.sum([K.count_params(w) for w in self.non_trainable_weights])
        total_count = trainable_count + non_trainable_count

        print('Total params: {:,}'.format(total_count))
        print('Trainable params: {:,}'.format(trainable_count))
        print('Non-trainable params: {:,}'.format(non_trainable_count))

        return total_count, trainable_count, non_trainable_count
    
    def print_epoch_statistics(self, params, summed_training_loss, summed_training_accuracy, message=None, require_result=False):
        if not params.verbose:
            if require_result:
                return self.evaluate(params, summed_training_loss, summed_training_accuracy)
            else:
                return
        
        loss, accuracy, val_loss, val_accuracy = self.evaluate(params, summed_training_loss, summed_training_accuracy)  

        if message is not None:
            print(message)
        
        print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy} - penalty: {self.get_regularization_penalty()}")
        hidden_layer_sizes = self.get_hidden_layer_sizes()
        print(f"hidden layer sizes: {hidden_layer_sizes}, total units: {sum(hidden_layer_sizes)}")
        if params.print_neurons:
            self.print_neurons()
        
        if require_result:
            return loss, accuracy, val_loss, val_accuracy
    
    def update_history(self, params, loss, accuracy, val_loss, val_accuracy):
        params.history['loss'].append(loss)
        params.history['accuracy'].append(accuracy)
        params.history['val_loss'].append(val_loss)
        params.history['val_accuracy'].append(val_accuracy)
        params.history['hidden_layer_sizes'].append(self.get_hidden_layer_sizes())
    
    @staticmethod
    def prepare_datasets(x, y, batch_size, validation_data):
        train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
        train_dataset = train_dataset.shuffle(buffer_size=20000).batch(batch_size)
        val_dataset = tf.data.Dataset.from_tensor_slices(validation_data).batch(batch_size)
        return train_dataset.prefetch(tf.data.AUTOTUNE), val_dataset.prefetch(tf.data.AUTOTUNE)
    
    def manage_dynamic_regularization(self, params, val_loss):
        if val_loss >= params.best_conditional_val_loss * params.stall_coefficient:
            # Training is currently in stall
            if not params.training_stalled:
                penalty = self.get_regularization_penalty() * params.regularization_penalty_multiplier
                print("Changing penalty...")
                # TODO this must be modified, penalty can differ for each layer
                self.set_regularization_penalty(penalty)
                params.training_stalled = True
        else:
            params.best_conditional_val_loss = val_loss
            params.training_stalled = False
    
    def grow_wrapper(self, params):
        dynamic_reqularization_active = params.regularization_penalty_multiplier != 1.
        if dynamic_reqularization_active:
            loss, accuracy, val_loss, val_accuracy = self.print_epoch_statistics(params, None, None, "Before growing:", require_result=True)
            self.manage_dynamic_regularization(params, val_loss)
        else:
            self.print_epoch_statistics(params, None, None, "Before growing:")

        self.grow(params)
        self.print_epoch_statistics(params, None, None, "After growing:")
    
    def prune_wrapper(self, params, summed_loss, summed_accuracy):
        loss, accuracy, _, _ = self.print_epoch_statistics(params, summed_loss, summed_accuracy, "Before pruning:", require_result=True)
        self.prune(params)
        _, _, val_loss, val_accuracy = self.print_epoch_statistics(params, None, None, "After pruning:", require_result=True)

        self.update_history(params, loss, accuracy, val_loss, val_accuracy)
    
    class ParameterContainer:
        def __init__(self, x, y, optimizer, batch_size, min_new_neurons, validation_data, pruning_threshold, 
                regularization_penalty_multiplier, stall_coefficient, growth_percentage, mini_epochs_per_epoch, verbose, print_neurons, use_static_graph):
            self.x = x
            self.y = y
            self.optimizer = optimizer
            self.batch_size = batch_size
            self.min_new_neurons = min_new_neurons
            self.validation_data = validation_data
            self.pruning_threshold = pruning_threshold
            self.regularization_penalty_multiplier = regularization_penalty_multiplier
            self.stall_coefficient = stall_coefficient
            self.growth_percentage = growth_percentage
            self.mini_epochs_per_epoch = mini_epochs_per_epoch
            self.verbose = verbose
            self.print_neurons = print_neurons
            self.use_static_graph = use_static_graph

            self.train_dataset, self.val_dataset = Sequential.prepare_datasets(x, y, batch_size, validation_data)
            self.history = self.prepare_history()

            self.best_conditional_val_loss = np.inf
            self.training_stalled = False
        
        @staticmethod
        def prepare_history():
            history = {
                'loss': list(),
                'accuracy': list(),
                'val_loss': list(),
                'val_accuracy': list(),
                'hidden_layer_sizes': list(),
            }
            return history
    
    def fit_single_step(self, params, x_batch, y_batch):
        with tf.GradientTape() as tape:
            y_pred = self(x_batch, training=True)
            raw_loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred)
            loss_value = tf.reduce_mean(raw_loss)
            loss_value += sum(self.losses)  # Add losses registered by model.add_loss

            loss = tf.reduce_sum(raw_loss)
            accuracy = float(tf.reduce_sum(tf.keras.metrics.sparse_categorical_accuracy(y_batch, y_pred)))

        grads = tape.gradient(loss_value, self.trainable_variables)
        params.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return loss, accuracy
    
    def fit_single_epoch(self, params):
        summed_loss = 0
        summed_accuracy = 0
        
        for mini_epoch in range(params.mini_epochs_per_epoch):
            summed_loss = 0
            summed_accuracy = 0

            if params.use_static_graph:
                fit_single_step_function = tf.function(self.fit_single_step)
            else:
                fit_single_step_function = self.fit_single_step
            for step, (x_batch, y_batch) in enumerate(params.train_dataset):
                loss, accuracy = fit_single_step_function(params, x_batch, y_batch)
                summed_loss += loss
                summed_accuracy += accuracy
        
        return summed_loss, summed_accuracy

    def fit(self, x, y, optimizer, schedule, batch_size, min_new_neurons, validation_data, pruning_threshold=0.001, regularization_penalty_multiplier=1., 
            stall_coefficient=1, growth_percentage=0.2, mini_epochs_per_epoch=1, verbose=True, print_neurons=False, use_static_graph=True):
        params = self.ParameterContainer(x=x, y=y, optimizer=optimizer, batch_size=batch_size, min_new_neurons=min_new_neurons, validation_data=validation_data, 
                                         pruning_threshold=pruning_threshold, regularization_penalty_multiplier=regularization_penalty_multiplier, stall_coefficient=stall_coefficient, 
                                         growth_percentage=growth_percentage, mini_epochs_per_epoch=mini_epochs_per_epoch, verbose=verbose, print_neurons=print_neurons, 
                                         use_static_graph=use_static_graph)
        self.build(x.shape)  # Necessary when verbose == False

        for epoch_no, epoch in enumerate(schedule):
            if verbose:
                print("##########################################################")
                print(f"Epoch {epoch_no + 1}/{len(schedule)}")
            
            self.set_regularization_penalty(epoch.regularization_penalty)
            self.set_regularization_method(epoch.regularization_method)

            if epoch.grow:
                self.grow_wrapper(params)
            
            summed_loss, summed_accuracy = self.fit_single_epoch(params)

            if epoch.prune:
                self.prune_wrapper(params, summed_loss, summed_accuracy)
            else:
                loss, accuracy, val_loss, val_accuracy = self.print_epoch_statistics(params, summed_loss, summed_accuracy, require_result=True)
                self.update_history(params, loss, accuracy, val_loss, val_accuracy)
        
        return params.history


################################################################################
# HELPER FUNCTIONS
################################################################################


def get_statistics_from_history(history):
    best_epoch_number = np.argmax(history['val_accuracy'])
    best_val_accuracy = history['val_accuracy'][best_epoch_number]
    best_hidden_layer_sizes = history['hidden_layer_sizes'][best_epoch_number]
    return best_val_accuracy, best_hidden_layer_sizes


def get_statistics_from_histories(histories):
    best_val_accuracies = list()
    all_best_hidden_layer_sizes = list()

    for history in histories:
        best_val_accuracy, best_hidden_layer_sizes = get_statistics_from_history(history)
        best_val_accuracies.append(best_val_accuracy)
        all_best_hidden_layer_sizes.append(best_hidden_layer_sizes)
    
    mean_best_val_accuracy = np.mean(best_val_accuracies)
    mean_best_hidden_layer_sizes = [np.mean(layer) for layer in list(zip(*all_best_hidden_layer_sizes))]
    
    return mean_best_val_accuracy, mean_best_hidden_layer_sizes


def cross_validate(train_fn, x, y, n_splits, random_state=42, *args, **kwargs):
    from sklearn.model_selection import KFold

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    histories = list()
    for i, (train_index, test_index) in enumerate(kf.split(x)):
        xtrain, xtest = x[train_index], x[test_index]
        ytrain, ytest = y[train_index], y[test_index]

        history = train_fn(xtrain, ytrain, validation_data=(xtest, ytest), *args, **kwargs)
        histories.append(history)

        best_val_accuracy, best_hidden_layer_sizes = get_statistics_from_history(history)
        print(f"Run {i} completed, best_val_accuracy: {best_val_accuracy}, best_hidden_layer_sizes: {best_hidden_layer_sizes}")

    mean_best_val_accuracy, mean_best_hidden_layer_sizes = get_statistics_from_histories(histories)
    print(f'mean_best_val_accuracy: {mean_best_val_accuracy}')
    print(f'mean_best_hidden_layer_sizes: {mean_best_hidden_layer_sizes}')

    return histories, mean_best_hidden_layer_sizes


def hyperparameter_search(train_fn, x, y, validation_data, *args, **kwargs):
    from itertools import product

    all_params = [*args] + list(kwargs.values())
    histories = list()

    best_overall_val_accuracy = -np.inf
    best_overall_combination = None

    for combination in product(*all_params):
        combination_args = combination[:len(args)]

        combination_kwargs_values = combination[len(args):]
        combination_kwargs = dict(zip(kwargs.keys(), combination_kwargs_values))

        history = train_fn(x, y, validation_data, *combination_args, **combination_kwargs)
        history['parameters'] = combination
        histories.append(history)

        best_val_accuracy, best_hidden_layer_sizes = get_statistics_from_history(history)
        print(f"Run with parameters {combination} completed, best_val_accuracy: {best_val_accuracy}, best_hidden_layer_sizes sizes: {best_hidden_layer_sizes}")

        if best_val_accuracy > best_overall_val_accuracy:
            best_overall_val_accuracy = best_val_accuracy
            best_overall_combination = combination
    
    print(f'Best overall combination: {best_overall_combination}, val_accuracy: {best_overall_val_accuracy}')

    return histories, best_overall_combination


def get_convolutional_model(x, layer_sizes, output_neurons=10):
    model = Sequential([
        Conv2D(layer_sizes[0], filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal', input_shape=x[0,:,:,:].shape),
        Conv2D(layer_sizes[1], filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.2),
        Conv2D(layer_sizes[2], filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal'),
        Conv2D(layer_sizes[3], filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.5),
        Flatten(),
        Dense(layer_sizes[4], activation='selu', kernel_initializer='lecun_normal'),
        Dense(output_neurons, activation='softmax', fixed_size=True),
    ])
    return model


def train_fn(x, y, validation_data, learning_rate, schedule, layer_sizes, output_neurons=10, min_new_neurons=20, 
             growth_percentage=0.2, verbose=False, use_static_graph=True):
    batch_size = 128

    model = get_convolutional_model(x, layer_sizes, output_neurons)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    history = model.fit(x=x, y=y, optimizer=optimizer, schedule=schedule, batch_size=batch_size, min_new_neurons=min_new_neurons, 
                        validation_data=validation_data, growth_percentage=growth_percentage, verbose=verbose, use_static_graph=use_static_graph)
    
    return history

# Final performance analysis

## Dynamic Auto-Sizing

In [ ]:
cifar100 = get_cifar_100_dataset()

169017344/169001437 [==============================] - 6s 0us/step


In [ ]:
%%time

schedule = Schedule([DynamicEpoch(0.00002, 'weighted_l1')] * 20 + [StaticEpochNoRegularization()] * 20)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0001, 0.0002, 0.0004], schedule=[schedule], layer_sizes=[[100, 100, 100, 100, 100]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0001, ec4ba8ef9e, [100, 100, 100, 100, 100], 100, 20, 0.2) completed, best_val_accuracy: 0.44, best_hidden_layer_sizes sizes: [90, 29, 38, 43, 100]
Run with parameters (0.0002, ec4ba8ef9e, [100, 100, 100, 100, 100], 100, 20, 0.2) completed, best_val_accuracy: 0.4448, best_hidden_layer_sizes sizes: [67, 23, 27, 73, 195]
Run with parameters (0.0004, ec4ba8ef9e, [100, 100, 100, 100, 100], 100, 20, 0.2) completed, best_val_accuracy: 0.4358, best_hidden_layer_sizes sizes: [60, 18, 55, 120, 452]
Best overall combination: (0.0002, ec4ba8ef9e, [100, 100, 100, 100, 100], 100, 20, 0.2), val_accuracy: 0.4448
CPU times: user 9min 57s, sys: 20.8 s, total: 10min 18s
Wall time: 9min 3s


In [ ]:
best_learning_rate = best_overall_combination[0]
best_learning_rate

0.0002

In [ ]:
%%time

schedule = Schedule([DynamicEpoch(0.00002, 'weighted_l1')] * 20 + [StaticEpochNoRegularization()] * 20)
histories, mean_best_hidden_layer_sizes = cross_validate(
    train_fn, cifar100.X_norm, cifar100.y, n_splits=6, learning_rate=best_learning_rate,
    schedule=schedule, layer_sizes=[100, 100, 100, 100, 100], output_neurons=100, min_new_neurons=20, growth_percentage=0.2
)

Run 0 completed, best_val_accuracy: 0.4428, best_hidden_layer_sizes: [68, 19, 33, 74, 191]
Run 1 completed, best_val_accuracy: 0.4512, best_hidden_layer_sizes: [62, 22, 29, 61, 195]
Run 2 completed, best_val_accuracy: 0.4484, best_hidden_layer_sizes: [67, 19, 30, 61, 200]
Run 3 completed, best_val_accuracy: 0.4485, best_hidden_layer_sizes: [67, 18, 33, 63, 199]
Run 4 completed, best_val_accuracy: 0.4577, best_hidden_layer_sizes: [62, 20, 27, 62, 190]
Run 5 completed, best_val_accuracy: 0.457, best_hidden_layer_sizes: [69, 23, 28, 66, 192]
mean_best_val_accuracy: 0.45093333333333335
mean_best_hidden_layer_sizes: [65.83333333333333, 20.166666666666668, 30.0, 64.5, 194.5]
CPU times: user 19min 26s, sys: 41.6 s, total: 20min 8s
Wall time: 17min 17s


In [3]:
np.std([0.4428, 0.4512, 0.4484, 0.4485, 0.4577, 0.457], ddof=1) * 100 * 2

1.1354411771054735

In [ ]:
mean_best_hidden_layer_sizes = [65.83333333333333, 20.166666666666668, 30.0, 64.5, 194.5]

In [ ]:
rounded_mean_best_hidden_layer_sizes = [round(x) for x in mean_best_hidden_layer_sizes]
rounded_mean_best_hidden_layer_sizes

[66, 20, 30, 64, 194]

## Static, no regularization

In [ ]:
%%time

schedule = Schedule([StaticEpochNoRegularization()] * 40)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0001, 0.0002, 0.0004], schedule=[schedule], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0001, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3175, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3379, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3427, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0004, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.3427
CPU times: user 7min 14s, sys: 30.5 s, total: 7min 45s
Wall time: 6min 3s


In [ ]:
%%time

schedule = Schedule([StaticEpochNoRegularization()] * 40)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0008], schedule=[schedule], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0008, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3319, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0008, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.3319
CPU times: user 2min 21s, sys: 10.3 s, total: 2min 32s
Wall time: 1min 55s


In [ ]:
%%time

schedule = Schedule([StaticEpochNoRegularization()] * 40)
histories, mean_best_hidden_layer_sizes = cross_validate(
    train_fn, cifar100.X_norm, cifar100.y, n_splits=6, learning_rate=0.0004,
    schedule=schedule, layer_sizes=[66, 20, 30, 64, 194], output_neurons=100, min_new_neurons=20, growth_percentage=0.2
)

Run 0 completed, best_val_accuracy: 0.3312, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 1 completed, best_val_accuracy: 0.3364, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 2 completed, best_val_accuracy: 0.3348, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 3 completed, best_val_accuracy: 0.3389, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 4 completed, best_val_accuracy: 0.336, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 5 completed, best_val_accuracy: 0.3362, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
mean_best_val_accuracy: 0.33558333333333334
mean_best_hidden_layer_sizes: [66.0, 20.0, 30.0, 64.0, 194.0]
CPU times: user 14min 2s, sys: 48.2 s, total: 14min 51s
Wall time: 11min 17s


In [ ]:
%%time

schedule = Schedule([StaticEpochNoRegularization()] * 40)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0001, 0.0002, 0.0004, 0.0008, 0.0016], schedule=[schedule], layer_sizes=[rounded_mean_best_hidden_layer_sizes], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0001, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3397, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3392, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.334, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0008, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3272, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0016, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3199, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0001, 4a0f172746, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.3397
CPU times: user 11min 56s, sys: 29 s, total: 12min

In [ ]:
best_learning_rate = best_overall_combination[0]
best_learning_rate

0.0001

In [ ]:
%%time

schedule = Schedule([StaticEpochNoRegularization()] * 40)
histories, mean_best_hidden_layer_sizes = cross_validate(
    train_fn, cifar100.X_norm, cifar100.y, n_splits=6, learning_rate=best_learning_rate,
    schedule=schedule, layer_sizes=rounded_mean_best_hidden_layer_sizes, output_neurons=100, min_new_neurons=20, growth_percentage=0.2
)

Run 0 completed, best_val_accuracy: 0.3183, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 1 completed, best_val_accuracy: 0.3265, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 2 completed, best_val_accuracy: 0.3134, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 3 completed, best_val_accuracy: 0.3345, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 4 completed, best_val_accuracy: 0.325, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 5 completed, best_val_accuracy: 0.3234, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
mean_best_val_accuracy: 0.3235166666666667
mean_best_hidden_layer_sizes: [66.0, 20.0, 30.0, 64.0, 194.0]
CPU times: user 14min 32s, sys: 34.6 s, total: 15min 7s
Wall time: 11min 33s


In [4]:
np.std([0.3183, 0.3265, 0.3134, 0.3345, 0.325, 0.3234], ddof=1) * 100 * 2

1.4458446205130986

## Pruning only, weighted L1 + fine-tuning

In [ ]:
%%time

schedule = Schedule([Epoch(False, True, 0.00002, 'weighted_l1')] * 20 + [StaticEpochNoRegularization()] * 20)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0001, 0.0002, 0.0004], schedule=[schedule], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0001, 758db6f544, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4388, best_hidden_layer_sizes sizes: [66, 20, 30, 51, 194]
Run with parameters (0.0002, 758db6f544, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4495, best_hidden_layer_sizes sizes: [60, 20, 29, 60, 194]
Run with parameters (0.0004, 758db6f544, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4394, best_hidden_layer_sizes sizes: [46, 20, 30, 64, 194]
Best overall combination: (0.0002, 758db6f544, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.4495
CPU times: user 8min 55s, sys: 22.3 s, total: 9min 18s
Wall time: 7min 18s


In [ ]:
%%time

schedule = Schedule([Epoch(False, True, 0.00002, 'weighted_l1')] * 20 + [StaticEpochNoRegularization()] * 20)
histories, mean_best_hidden_layer_sizes = cross_validate(
    train_fn, cifar100.X_norm, cifar100.y, n_splits=6, learning_rate=0.0002,
    schedule=schedule, layer_sizes=[66, 20, 30, 64, 194], output_neurons=100, min_new_neurons=20, growth_percentage=0.2
)

Run 0 completed, best_val_accuracy: 0.45, best_hidden_layer_sizes: [63, 20, 30, 58, 194]
Run 1 completed, best_val_accuracy: 0.4486, best_hidden_layer_sizes: [62, 20, 30, 64, 194]
Run 2 completed, best_val_accuracy: 0.4473, best_hidden_layer_sizes: [60, 20, 30, 62, 194]
Run 3 completed, best_val_accuracy: 0.4588, best_hidden_layer_sizes: [60, 20, 30, 60, 194]
Run 4 completed, best_val_accuracy: 0.4508, best_hidden_layer_sizes: [60, 20, 30, 63, 194]
Run 5 completed, best_val_accuracy: 0.4444, best_hidden_layer_sizes: [61, 20, 30, 63, 194]
mean_best_val_accuracy: 0.44998333333333335
mean_best_hidden_layer_sizes: [61.0, 20.0, 30.0, 61.666666666666664, 194.0]
CPU times: user 17min 32s, sys: 42.3 s, total: 18min 14s
Wall time: 14min 18s


In [5]:
np.std([0.45, 0.4486, 0.4473, 0.4588, 0.4508, 0.4444], ddof=1) * 100 * 2

0.9744263269568737

## Pruning only, weighted L1

In [ ]:
%%time

schedule = Schedule([Epoch(False, True, 0.00002, 'weighted_l1')] * 40)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0001, 0.0002, 0.0004], schedule=[schedule], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0001, 865b7710ae, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4217, best_hidden_layer_sizes sizes: [63, 20, 30, 43, 142]
Run with parameters (0.0002, 865b7710ae, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.437, best_hidden_layer_sizes sizes: [52, 20, 29, 63, 194]
Run with parameters (0.0004, 865b7710ae, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4319, best_hidden_layer_sizes sizes: [37, 20, 27, 64, 194]
Best overall combination: (0.0002, 865b7710ae, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.437
CPU times: user 10min 45s, sys: 26.9 s, total: 11min 12s
Wall time: 8min 58s


In [ ]:
%%time

schedule = Schedule([Epoch(False, True, 0.00002, 'weighted_l1')] * 40)
histories, mean_best_hidden_layer_sizes = cross_validate(
    train_fn, cifar100.X_norm, cifar100.y, n_splits=6, learning_rate=0.0002,
    schedule=schedule, layer_sizes=[66, 20, 30, 64, 194], output_neurons=100, min_new_neurons=20, growth_percentage=0.2
)

Run 0 completed, best_val_accuracy: 0.4338, best_hidden_layer_sizes: [46, 20, 28, 64, 194]
Run 1 completed, best_val_accuracy: 0.4405, best_hidden_layer_sizes: [48, 20, 29, 63, 194]
Run 2 completed, best_val_accuracy: 0.4322, best_hidden_layer_sizes: [44, 20, 27, 63, 194]
Run 3 completed, best_val_accuracy: 0.44, best_hidden_layer_sizes: [47, 20, 26, 62, 193]
Run 4 completed, best_val_accuracy: 0.4413, best_hidden_layer_sizes: [45, 20, 29, 61, 194]
Run 5 completed, best_val_accuracy: 0.4355, best_hidden_layer_sizes: [47, 20, 27, 60, 194]
mean_best_val_accuracy: 0.43721666666666675
mean_best_hidden_layer_sizes: [46.166666666666664, 20.0, 27.666666666666668, 62.166666666666664, 193.83333333333334]
CPU times: user 21min 10s, sys: 52 s, total: 22min 2s
Wall time: 17min 41s


In [6]:
np.std([0.4338, 0.4405, 0.4322, 0.44, 0.4413, 0.4355], ddof=1) * 100 * 2

0.7745364204907796

## Static, weighted L1 + fine-tuning

In [ ]:
%%time

schedule = Schedule([StaticEpoch(0.00002, 'weighted_l1')] * 20 + [StaticEpochNoRegularization()] * 20)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0001, 0.0002, 0.0004], schedule=[schedule], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0001, e7901b090a, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3984, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, e7901b090a, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4299, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, e7901b090a, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.436, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0004, e7901b090a, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.436
CPU times: user 7min 48s, sys: 27.9 s, total: 8min 16s
Wall time: 6min 18s


In [ ]:
%%time

schedule = Schedule([StaticEpoch(0.00002, 'weighted_l1')] * 20 + [StaticEpochNoRegularization()] * 20)
histories, mean_best_hidden_layer_sizes = cross_validate(
    train_fn, cifar100.X_norm, cifar100.y, n_splits=6, learning_rate=0.0004,
    schedule=schedule, layer_sizes=[66, 20, 30, 64, 194], output_neurons=100, min_new_neurons=20, growth_percentage=0.2
)

Run 0 completed, best_val_accuracy: 0.4288, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 1 completed, best_val_accuracy: 0.4404, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 2 completed, best_val_accuracy: 0.4328, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 3 completed, best_val_accuracy: 0.443, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 4 completed, best_val_accuracy: 0.45, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 5 completed, best_val_accuracy: 0.4344, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
mean_best_val_accuracy: 0.4382333333333334
mean_best_hidden_layer_sizes: [66.0, 20.0, 30.0, 64.0, 194.0]
CPU times: user 15min, sys: 39.6 s, total: 15min 40s
Wall time: 11min 58s


In [7]:
np.std([0.4288, 0.4404, 0.4328, 0.443, 0.45, 0.4344], ddof=1) * 100 * 2

1.5467988449267298

## Static, weighted L1

In [ ]:
%%time

schedule = Schedule([StaticEpoch(0.00002, 'weighted_l1')] * 40)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0001, 0.0002, 0.0004], schedule=[schedule], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0001, f4be7cd642, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3692, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, f4be7cd642, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.406, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, f4be7cd642, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4305, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0004, f4be7cd642, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.4305
CPU times: user 8min 16s, sys: 27.1 s, total: 8min 43s
Wall time: 6min 40s


In [ ]:
%%time

schedule = Schedule([StaticEpoch(0.00002, 'weighted_l1')] * 40)
histories, mean_best_hidden_layer_sizes = cross_validate(
    train_fn, cifar100.X_norm, cifar100.y, n_splits=6, learning_rate=0.0004,
    schedule=schedule, layer_sizes=[66, 20, 30, 64, 194], output_neurons=100, min_new_neurons=20, growth_percentage=0.2
)

Run 0 completed, best_val_accuracy: 0.4171, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 1 completed, best_val_accuracy: 0.437, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 2 completed, best_val_accuracy: 0.4311, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 3 completed, best_val_accuracy: 0.4382, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 4 completed, best_val_accuracy: 0.4359, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 5 completed, best_val_accuracy: 0.4218, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
mean_best_val_accuracy: 0.43018333333333336
mean_best_hidden_layer_sizes: [66.0, 20.0, 30.0, 64.0, 194.0]
CPU times: user 15min 54s, sys: 44.7 s, total: 16min 38s
Wall time: 12min 45s


In [8]:
np.std([0.4171, 0.437, 0.4311, 0.4382, 0.4359, 0.4218], ddof=1) * 100 * 2

1.7566065770873847

## Static, L1

In [ ]:
schedule1 = Schedule([StaticEpoch(0.002, 'l1')] * 40)
schedule2 = Schedule([StaticEpoch(0.0002, 'l1')] * 40)
schedule3 = Schedule([StaticEpoch(0.00002, 'l1')] * 40)
schedule1, schedule2, schedule3

(b4b64f7e12, 50d84757a6, f6a1bb91c8)

In [ ]:
%%time

schedule = Schedule([StaticEpoch(0.00002, 'l1')] * 40)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0002], schedule=[schedule1, schedule2, schedule3], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0002, b4b64f7e12, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.2492, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, 50d84757a6, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4198, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, f6a1bb91c8, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.341, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0002, 50d84757a6, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.4198
CPU times: user 7min 42s, sys: 20 s, total: 8min 2s
Wall time: 6min 7s


In [ ]:
schedule1 = Schedule([StaticEpoch(0.0008, 'l1')] * 40)
schedule2 = Schedule([StaticEpoch(0.0004, 'l1')] * 40)
schedule3 = Schedule([StaticEpoch(0.0002, 'l1')] * 40)
schedule4 = Schedule([StaticEpoch(0.0001, 'l1')] * 40)
schedule5 = Schedule([StaticEpoch(0.00005, 'l1')] * 40)
schedule1, schedule2, schedule3, schedule4, schedule5

(0970d93389, 2bac11ab4e, 50d84757a6, 3bdfd81ddb, 8e0e7c7bed)

In [ ]:
%%time

histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0002], schedule=[schedule1, schedule2, schedule3, schedule4, schedule5], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0002, 0970d93389, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3839, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4209, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, 50d84757a6, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4058, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, 3bdfd81ddb, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3742, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, 8e0e7c7bed, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3454, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0002, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.4209
CPU times: user 12min 58s, sys: 34 s, total: 13mi

In [ ]:
%%time

schedule = Schedule([StaticEpoch(0.0004, 'l1')] * 40)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.00005, 0.0001, 0.0002, 0.0004, 0.0008], schedule=[schedule], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (5e-05, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.346, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0001, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3759, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4198, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4476, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0008, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.434, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0004, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.4476
CPU times: user 13min 3s, sys: 38.6 s, total: 13min 

In [ ]:
%%time

schedule = Schedule([StaticEpoch(0.0004, 'l1')] * 40)
histories, mean_best_hidden_layer_sizes = cross_validate(
    train_fn, cifar100.X_norm, cifar100.y, n_splits=6, learning_rate=0.0004,
    schedule=schedule, layer_sizes=[66, 20, 30, 64, 194], output_neurons=100, min_new_neurons=20, growth_percentage=0.2
)

Run 0 completed, best_val_accuracy: 0.4381, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 1 completed, best_val_accuracy: 0.4375, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 2 completed, best_val_accuracy: 0.4278, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 3 completed, best_val_accuracy: 0.4453, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 4 completed, best_val_accuracy: 0.446, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 5 completed, best_val_accuracy: 0.4314, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
mean_best_val_accuracy: 0.43768333333333337
mean_best_hidden_layer_sizes: [66.0, 20.0, 30.0, 64.0, 194.0]
CPU times: user 15min 49s, sys: 45.6 s, total: 16min 35s
Wall time: 12min 41s


In [9]:
np.std([0.4381, 0.4375, 0.4278, 0.4453, 0.446, 0.4314], ddof=1) * 100 * 2

1.454780624928261

## Static, L1 + fine-tuning

In [ ]:
schedule1 = Schedule([StaticEpoch(0.0016, 'l1')] * 40)
schedule2 = Schedule([StaticEpoch(0.0008, 'l1')] * 40)
schedule3 = Schedule([StaticEpoch(0.0004, 'l1')] * 40)
schedule4 = Schedule([StaticEpoch(0.0002, 'l1')] * 40)
schedule5 = Schedule([StaticEpoch(0.0001, 'l1')] * 40)
schedule1, schedule2, schedule3, schedule4, schedule5

(40c4b8067f, 0970d93389, 2bac11ab4e, 50d84757a6, 3bdfd81ddb)

In [ ]:
%%time

histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0004], schedule=[schedule1, schedule2, schedule3, schedule4, schedule5], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0004, 40c4b8067f, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3189, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, 0970d93389, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.393, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4528, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, 50d84757a6, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4493, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, 3bdfd81ddb, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3916, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0004, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.4528
CPU times: user 13min 12s, sys: 39.8 s, total: 13m

In [ ]:
%%time

schedule = Schedule([StaticEpoch(0.0004, 'l1')] * 40)
histories, best_overall_combination = hyperparameter_search(train_fn, x=cifar100.X_train_norm, y=cifar100.y_train, validation_data=(cifar100.X_test_norm, cifar100.y_test), 
                                  learning_rate=[0.0001, 0.0002, 0.0004, 0.0008], schedule=[schedule], layer_sizes=[[66, 20, 30, 64, 194]], 
                                  output_neurons=[100], min_new_neurons=[20], growth_percentage=[0.2])

Run with parameters (0.0001, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.3903, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0002, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.429, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0004, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4493, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Run with parameters (0.0008, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2) completed, best_val_accuracy: 0.4072, best_hidden_layer_sizes sizes: [66, 20, 30, 64, 194]
Best overall combination: (0.0004, 2bac11ab4e, [66, 20, 30, 64, 194], 100, 20, 0.2), val_accuracy: 0.4493
CPU times: user 10min 32s, sys: 31.5 s, total: 11min 3s
Wall time: 8min 25s


In [ ]:
%%time

schedule = Schedule([StaticEpoch(0.0004, 'l1')] * 40)
histories, mean_best_hidden_layer_sizes = cross_validate(
    train_fn, cifar100.X_norm, cifar100.y, n_splits=6, learning_rate=0.0004,
    schedule=schedule, layer_sizes=[66, 20, 30, 64, 194], output_neurons=100, min_new_neurons=20, growth_percentage=0.2
)

Run 0 completed, best_val_accuracy: 0.4439, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 1 completed, best_val_accuracy: 0.4489, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 2 completed, best_val_accuracy: 0.4403, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 3 completed, best_val_accuracy: 0.4366, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 4 completed, best_val_accuracy: 0.4391, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
Run 5 completed, best_val_accuracy: 0.4395, best_hidden_layer_sizes: [66, 20, 30, 64, 194]
mean_best_val_accuracy: 0.4413833333333333
mean_best_hidden_layer_sizes: [66.0, 20.0, 30.0, 64.0, 194.0]
CPU times: user 15min 45s, sys: 44.7 s, total: 16min 30s
Wall time: 12min 38s
